In [ ]:
# 1) Imports
import sqlite3
import pandas as pd
from sqlalchemy import create_engine, MetaData
from sqlalchemy.exc import SQLAlchemyError

# 2) Configuración de conexiones
sqlite_path = "db.sqlite3"  # tu archivo SQLite
# URL de Supabase con psycopg2
pg_url = (
    "postgresql+psycopg2://postgres:LAR81097sj12lp*"
    "@db.gurdmtceqmebgrpcrzft.supabase.co:5432/postgres"
)

# Engine solo para Postgres
pg_engine = create_engine(pg_url, connect_args={"sslmode": "require"})

# 3) Lee la lista de tablas de SQLite (usando sqlite3)
sqlite_conn = sqlite3.connect(sqlite_path)
cursor = sqlite_conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [row[0] for row in cursor.fetchall() if not row[0].startswith("sqlite_")]
cursor.close()

# 4) Migra cada tabla
for table_name in tables:
    try:
        print(f"Migrando tabla: {table_name}…")
        # Usamos sqlite3.Connection, que sí tiene cursor()
        df = pd.read_sql_query(f'SELECT * FROM "{table_name}"', sqlite_conn)

        # Volcamos al Postgres de Supabase
        df.to_sql(
            name=table_name,
            con=pg_engine,
            if_exists="replace",  # o 'append' si prefieres
            index=False
        )
        print(f"✅ '{table_name}' migrada correctamente.")
    except SQLAlchemyError as e:
        print(f"❌ Error migrando '{table_name}': {e}")

sqlite_conn.close()
print("🏁 ¡Migración completada!")


In [4]:
# export_sqlite_to_csvs.py

import sqlite3
import pandas as pd
from pathlib import Path

# 1) Rutas y directorios
DB_PATH    = "db.sqlite3"                # Ajusta si tu BD está en otra ruta
EXPORT_DIR = Path("data")         # Carpeta donde se guardarán los CSVs
EXPORT_DIR.mkdir(exist_ok=True)

# 2) Conexión a SQLite y listado de tablas
conn   = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [row[0] for row in cursor.fetchall() if not row[0].startswith("sqlite_")]
cursor.close()

# 3) Exportar cada tabla
for table in tables:
    print(f"→ Exportando tabla: {table}")
    df = pd.read_sql_query(f'SELECT * FROM "{table}"', conn)
    out = EXPORT_DIR / f"{table}.csv"
    df.to_csv(out, index=False)
    print(f"   ✔ Guardado en {out}")

conn.close()
print("\n✅ ¡Todos los CSVs generados en ./csv_exports/ !")


→ Exportando tabla: django_migrations
   ✔ Guardado en data\django_migrations.csv
→ Exportando tabla: landing_requestcontactmodel
   ✔ Guardado en data\landing_requestcontactmodel.csv
→ Exportando tabla: auth_group_permissions
   ✔ Guardado en data\auth_group_permissions.csv
→ Exportando tabla: auth_user_groups
   ✔ Guardado en data\auth_user_groups.csv
→ Exportando tabla: auth_user_user_permissions
   ✔ Guardado en data\auth_user_user_permissions.csv
→ Exportando tabla: django_admin_log
   ✔ Guardado en data\django_admin_log.csv
→ Exportando tabla: django_content_type
   ✔ Guardado en data\django_content_type.csv
→ Exportando tabla: auth_permission
   ✔ Guardado en data\auth_permission.csv
→ Exportando tabla: auth_group
   ✔ Guardado en data\auth_group.csv
→ Exportando tabla: auth_user
   ✔ Guardado en data\auth_user.csv
→ Exportando tabla: django_session
   ✔ Guardado en data\django_session.csv
→ Exportando tabla: landing_reseniamodel
   ✔ Guardado en data\landing_reseniamodel.csv
→ 